In [1]:
from plot_params import *
import pandas as pd
df=pd.read_csv(outdir / 'Human_annotated_30bp.csv.gz')

In [2]:
df['subseq'].str.len().min()

30

In [3]:
df['subseq'].str.len().max()

30

In [4]:
df.head()

,Unnamed: 0,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,start_annotation,end_annotation,subseq
0,0,ENST00000000233,0,30,0.000000,0.044434,UTR5,UTR5,CTGCTGCTGCTGCGCCCCATCCCCCCGCGG
1,1,ENST00000000233,15,45,0.000000,0.044434,UTR5,UTR5,CCCATCCCCCCGCGGCCGGCCAGTTCCAGC
2,2,ENST00000000233,30,60,0.000000,0.044434,UTR5,UTR5,CCGGCCAGTTCCAGCCCGCACCCCGCGTCG
3,3,ENST00000000233,45,75,0.000000,0.044434,UTR5,UTR5,CCGCACCCCGCGTCGGTGCCCGCGCCCCTC
4,4,ENST00000000233,60,90,-0.001953,0.042480,UTR5,CDS,GTGCCCGCGCCCCTCCCCGGGCCCCGCCAT


In [5]:
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq
def df2fasta(df, fname, col='subseq', id='Unnamed: 0'):
    
    
    sequences = []
    for index, row in df.iterrows():
        sequences.append(SeqRecord(Seq(row[col]), str(row[id]), "", ""))
    with open(fname, "w") as output_handle:
        SeqIO.write(sequences, output_handle, "fasta")


In [6]:
outdir

PosixPath('/tscc/nfs/home/hsher/ps-yeolab5/karen_data')

In [7]:
df2fasta(df, outdir / 'all_human_30bp.fa')

In [8]:
df['annotation']=df.apply(
    lambda row: 'UTR5' if row['start_annotation']=='UTR5' and row['end_annotation']=='UTR5' else
     'UTR3' if row['start_annotation']=='UTR3' and row['end_annotation']=='UTR3' else 'CDS', axis = 1)

In [9]:
df2fasta(df.loc[(df['annotation']=='UTR3')&(df['difference']<-0.1)], 
         outdir / 'utr3_forefround_human_30bp.fa')
df2fasta(df.loc[(df['annotation']=='UTR3')&(df['difference']>0.1)], 
         outdir / 'utr3_backfround_human_30bp.fa')

df2fasta(df.loc[(df['annotation']=='UTR5')&(df['difference']<-0.1)], 
         outdir / 'utr5_forefround_human_30bp.fa')
df2fasta(df.loc[(df['annotation']=='UTR5')&(df['difference']>0.1)], 
         outdir / 'utr5_backfround_human_30bp.fa')

In [10]:
df.loc[(df['annotation']=='UTR5')&(df['difference']<-0.1)]

,Unnamed: 0,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,start_annotation,end_annotation,subseq,annotation
342,342,ENST00000002165,0,60,-0.419922,0.271484,UTR5,UTR5,GCGGCATACTCAGCGGCAGAGACGGAAGAACAGCGCTCCCGAGGCC...,UTR5
1560,1560,ENST00000005259,0,60,-0.512695,0.135742,UTR5,UTR5,GTCCCGCGCGTCGGCGGCCGCGGAGCAGCGCAGGGAGCCAGGCGGG...,UTR5
6570,6570,ENST00000035307,990,1050,-0.102539,0.126953,UTR5,UTR5,AGCCCCTTCAGTAGCCCGCCTGAGGACCGATGCCAGAGGCAGGCAT...,UTR5
6571,6571,ENST00000035307,1020,1080,-0.105469,0.124023,UTR5,UTR5,TGCCAGAGGCAGGCATTCTTCCGGAAAGGCCCACTGAGGCAGGCTC...,UTR5
6572,6572,ENST00000035307,1050,1110,-0.172119,0.057373,UTR5,UTR5,CCACTGAGGCAGGCTCCGGCTCCTCTGGTTGGGGCTGTTGTTTTGA...,UTR5
...,...,...,...,...,...,...,...,...,...,...
3460885,3460885,ENST00000704699,390,450,-0.136719,0.371094,UTR5,UTR5,AAGAAAGAAAACAAAGAAAAACATACTCCAGAATTCCTAATAGAAC...,UTR5
3461524,3461524,ENST00000704702,270,330,-0.125000,0.150391,UTR5,UTR5,CGCGTGTGCCTCGCGCCGGTCCACGCGGGGAGAGCACTGGGGACCG...,UTR5
3461525,3461525,ENST00000704702,300,360,-0.110352,0.165039,UTR5,UTR5,AGAGCACTGGGGACCGAGACCCGGCACCACCTCCCGGTCCGCCCTC...,UTR5
3461535,3461535,ENST00000704702,600,660,-0.213135,0.062256,UTR5,UTR5,ACTGGAACTGTCTGATTAAGAAAAACAGAATAATTCTGAAAGAAAG...,UTR5


# Commands to run HOMER

```
cd /tscc/nfs/home/hsher/ps-yeolab5/karen_data
findMotifs.pl utr3_forefround_human.fa fasta utr3_homer_results_human/ -fasta utr3_backfround_human.fa \
    -rna -nofacts
findMotifs.pl utr5_forefround_human.fa fasta utr5_homer_results_human/ -fasta utr5_backfround_human.fa \
    -rna -nofacts
```